<a href="https://colab.research.google.com/github/cmannnn/practical_deep_learning_for_coders/blob/main/ch_4_digit_recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MNIST digit recognizer

In [2]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

Mounted at /content/gdrive


In [3]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

##Q1

Create your own implementation of Learner from scratch, based on the training loop shown in this chapter.

In [4]:
# setting the path to the MNIST dataset
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

In [7]:
# grabbing just the threes and sevens form the training set to use
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

In [8]:
# opening the training 3's/7's and changing them to tensors
three_tensor = [tensor(Image.open(o)) for o in threes]
seven_tensor = [tensor(Image.open(o)) for o in sevens]

In [9]:
# creating Pytorch tensors of the valid 3s and 7s
valid_3_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'3').ls()])

valid_7_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'7').ls()])

In [10]:
# stacking three_tensor  normalize the train 3 tensor
stacked_threes = torch.stack(three_tensor).float()/255

# stack then normalize the train 7 tensor
stacked_sevens = torch.stack(seven_tensor).float()/255

# normalizing the valid 3 tensor (valid_3_tens already stacked)
valid_3_tens = valid_3_tens.float()/255

# normalizing the valid 7 tensor (valid_3_tens already stacked)
valid_7_tens = valid_7_tens.float()/255

In [11]:
# changing the training rank 3 tensors to a rank 2 tensor using view
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)

# changing the testing rank 3 tensors to a rank 2 tensor using view
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)

In [13]:
# creating labels for our training data as a column vector
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)

# creating labels for our testing data as a column vector
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)

In [14]:
# zipping the x, y training dataset
dset = list(zip(train_x, train_y))
dl = DataLoader(dset, batch_size = 256)

# zipping the x, y training dataset
valid_dset = list(zip(valid_x, valid_y))
valid_dl = DataLoader(valid_dset, batch_size = 256)

# loading training DataLoader and testing DataLoader
ds = DataLoader(dl, valid_dl)

In [16]:
# creating the neural net
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1),
    nn.Sigmoid()
)

In [ ]:
# PyTorch binary loss function?

In [ ]:
class BasicOptim:
  def __init__(self,params,lr): self.params, self.lr = list(params),lr

  

##Q2

Complete all the steps in this chapter using the full MNIST datasets (that is, for all digits, not just 3s and 7s). This is a significant project and will take you quite a bit of time to complete! 